In [ ]:
import numpy as np
import pandas as pd

barnstable_data = pd.read_excel("barnstable_analysis_data.xlsx", sheet_name = "data")

### 1 Summary Statistics

### 2 Probability of Underperformance

### 3 Full Sample Analysis

### 4 IS Estimate & OOS Likelihood